In [1]:
#!unzip ahalytix_internship_hiring_ai_challenge-dataset.zip

In [3]:

import  re
import nltk
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error as MSE, r2_score
from scipy.stats import skew,kurtosis,zscore
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [4]:
df = pd.read_csv("TRAIN.csv")
df.head(5)

,date_time,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [5]:
df.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,38563.000000,38563.000000,38563.000000,38563.000000,38563.000000
mean,281.351757,0.392733,0.000278,49.920364,3260.940409
std,13.216927,50.075055,0.009131,38.849106,1991.628329
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.858000,0.000000,0.000000,1.000000,1186.500000
50%,282.750000,0.000000,0.000000,64.000000,3378.000000
75%,291.540000,0.000000,0.000000,90.000000,4939.000000
max,308.240000,9831.300000,0.510000,100.000000,7280.000000


In [6]:
df.traffic_volume.skew()

-0.08683588462501615

In [7]:
df.holiday.unique()

array(['None', 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'New Years Day', 'Washingtons Birthday',
       'Memorial Day', 'Independence Day', 'State Fair', 'Labor Day',
       'Martin Luther King Jr Day'], dtype=object)

In [8]:
df.holiday.value_counts()

None                         38515
Thanksgiving Day                 5
Labor Day                        5
Columbus Day                     5
New Years Day                    5
Christmas Day                    5
Veterans Day                     4
Washingtons Birthday             4
Independence Day                 4
State Fair                       4
Memorial Day                     4
Martin Luther King Jr Day        3
Name: holiday, dtype: int64

In [9]:
df['date_time'] = pd.to_datetime(df.date_time)
df['weekday'] = df.date_time.dt.weekday
df['date'] = df.date_time.dt.date
df['hour'] = df.date_time.dt.hour
df['month'] = df.date_time.dt.month
df['year'] = df.date_time.dt.year
df = df.drop(["date_time","date"],1)
df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume,weekday,hour,month,year
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,5545,1,9,10,2012
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,4516,1,10,10,2012
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,4767,1,11,10,2012
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,5026,1,12,10,2012
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,4918,1,13,10,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
38558,None,274.35,0.0,0.0,90,Snow,light snow,4297,2,18,11,2017
38559,None,274.62,0.0,0.0,90,Drizzle,light intensity drizzle,3045,2,19,11,2017
38560,None,274.62,0.0,0.0,90,Mist,mist,3045,2,19,11,2017
38561,None,274.62,0.0,0.0,90,Rain,light rain,3045,2,19,11,2017


In [10]:
df_text = df[["holiday","weather_main","weather_description"]]
df_text.head(2)

,holiday,weather_main,weather_description
0,None,Clouds,scattered clouds
1,None,Clouds,broken clouds


In [11]:
df_num = df.drop(["holiday","weather_main","weather_description","traffic_volume"],1)
df_num.head(4)

,temp,rain_1h,snow_1h,clouds_all,weekday,hour,month,year
0,288.28,0.0,0.0,40,1,9,10,2012
1,289.36,0.0,0.0,75,1,10,10,2012
2,289.58,0.0,0.0,90,1,11,10,2012
3,290.13,0.0,0.0,90,1,12,10,2012


In [12]:
#df_num['holiday'] = np.where(((df_num.holiday != "None")),'Misc',df_num.holiday)

In [13]:
#df_num.holiday.value_counts()

In [14]:
#dummies = pd.get_dummies(df_num['holiday'], prefix = 'holiday')
#df_num = df_num.drop(['holiday'],1)

In [15]:
#df_num = pd.concat([df_num, dummies], 1)
#df_num.head(2)

In [16]:
y= df.traffic_volume

In [17]:
X_num = df_num

##using NLP

In [18]:
df_text['main'] = df_text[df_text.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

df_text.main.head(3)

C:\Users\welcome\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    None,Clouds,scattered clouds
1       None,Clouds,broken clouds
2     None,Clouds,overcast clouds
Name: main, dtype: object

In [19]:
X_text = df_text.main
X_text.head(2)

0    None,Clouds,scattered clouds
1       None,Clouds,broken clouds
Name: main, dtype: object

In [20]:
model = []
for i in range(0, df.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [21]:
model[1]

'none clouds broken clouds'

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\welcome\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
vector = TfidfVectorizer(max_features = 4, min_df = 1, max_df = 0.8, stop_words = stopwords.words('english'),ngram_range = (2,2))
X_text = vector.fit_transform(model).toarray()

In [24]:
X_text

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [25]:
X = np.concatenate((X_text,X_num),axis = 1)
X

array([[0.000e+00, 0.000e+00, 1.000e+00, ..., 9.000e+00, 1.000e+01,
        2.012e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.000e+01, 1.000e+01,
        2.012e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.100e+01, 1.000e+01,
        2.012e+03],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.900e+01, 1.100e+01,
        2.017e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.900e+01, 1.100e+01,
        2.017e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+01, 1.100e+01,
        2.017e+03]])

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 42)

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
regr = RandomForestRegressor(n_jobs = 4 ,random_state = 0 ,n_estimators = 100,verbose =1 )

In [29]:
regr.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.5s finished


RandomForestRegressor(n_jobs=4, random_state=0, verbose=1)

In [30]:
import numpy as np

pred1 = regr.predict(X_test) 
rmse = np.sqrt(MSE(y_test, pred1)) 
print("RMSE : % f" %(rmse)) 

RMSE :  379.059312


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [31]:
r2_score(y_test,pred1)*100

96.43514689713302

In [32]:
pred1[:10]

array([5734.74, 3058.2 ,  517.22, 2308.17, 4327.7 , 4488.63, 3136.54,
       5374.02, 5304.28, 4595.92])

In [33]:
y_g = np.array(y_test)
y_g[:10]

array([5754, 2685,  496, 2157, 4542, 4609, 3349, 3716, 5168, 4593],
      dtype=int64)

Using LightGBM

##Fitting on test data

In [36]:
df = pd.read_csv("TEST.csv")
df.head(5)

,date_time,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description
0,2017-11-01 20:00:00,None,274.75,0.0,0.0,90,Mist,mist
1,2017-11-01 20:00:00,None,274.75,0.0,0.0,90,Rain,light rain
2,2017-11-01 21:00:00,None,274.79,0.0,0.0,90,Mist,mist
3,2017-11-01 21:00:00,None,274.79,0.0,0.0,90,Drizzle,drizzle
4,2017-11-01 22:00:00,None,275.08,0.0,0.0,90,Mist,mist


In [37]:
df['date_time'] = pd.to_datetime(df.date_time)
df['weekday'] = df.date_time.dt.weekday
df['date'] = df.date_time.dt.date
df['hour'] = df.date_time.dt.hour
df['month'] = df.date_time.dt.month
df['year'] = df.date_time.dt.year
df = df.drop(["date_time","date"],1)
df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,weekday,hour,month,year
0,None,274.75,0.0,0.0,90,Mist,mist,2,20,11,2017
1,None,274.75,0.0,0.0,90,Rain,light rain,2,20,11,2017
2,None,274.79,0.0,0.0,90,Mist,mist,2,21,11,2017
3,None,274.79,0.0,0.0,90,Drizzle,drizzle,2,21,11,2017
4,None,275.08,0.0,0.0,90,Mist,mist,2,22,11,2017
...,...,...,...,...,...,...,...,...,...,...,...
9636,None,283.45,0.0,0.0,75,Clouds,broken clouds,6,19,9,2018
9637,None,282.76,0.0,0.0,90,Clouds,overcast clouds,6,20,9,2018
9638,None,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,6,21,9,2018
9639,None,282.09,0.0,0.0,90,Clouds,overcast clouds,6,22,9,2018


In [38]:
df_text = df[["holiday","weather_main","weather_description"]]
df_text.head(2)

,holiday,weather_main,weather_description
0,None,Mist,mist
1,None,Rain,light rain


In [39]:
df_num = df.drop(["holiday","weather_main","weather_description"],1)
df_num.head(4)

,temp,rain_1h,snow_1h,clouds_all,weekday,hour,month,year
0,274.75,0.0,0.0,90,2,20,11,2017
1,274.75,0.0,0.0,90,2,20,11,2017
2,274.79,0.0,0.0,90,2,21,11,2017
3,274.79,0.0,0.0,90,2,21,11,2017


In [40]:
X_num = df_num

In [41]:
df_text['main'] = df_text[df_text.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

df_text.main.head(3)

C:\Users\welcome\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0          None,Mist,mist
1    None,Rain,light rain
2          None,Mist,mist
Name: main, dtype: object

In [42]:
X_text = df_text.main
X_text.head(2)

0          None,Mist,mist
1    None,Rain,light rain
Name: main, dtype: object

In [43]:
model = []
for i in range(0, df.shape[0]):
    data = re.sub(r'\W', ' ', str(X_text[i]))
    data = data.lower()
    data = re.sub(r'^br$', ' ', data)
    data = re.sub(r'\s+br\s+',' ',data)
    data = re.sub(r'\s+[a-z]\s+', ' ',data)
    data = re.sub(r'^b\s+', '', data)
    data = re.sub(r'\s+', ' ', data)
    model.append(data)   

In [44]:
vector = TfidfVectorizer(max_features = 4, min_df = 1, max_df = 0.8, stop_words = stopwords.words('english'),ngram_range = (2,2))
X_text = vector.fit_transform(model).toarray()

In [45]:
X = np.concatenate((X_text,X_num),axis = 1)
X

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+01, 1.100e+01,
        2.017e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+01, 1.100e+01,
        2.017e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.100e+01, 1.100e+01,
        2.017e+03],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.100e+01, 9.000e+00,
        2.018e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.200e+01, 9.000e+00,
        2.018e+03],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.300e+01, 9.000e+00,
        2.018e+03]])

In [46]:
predictions = regr.predict(X)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [47]:
predictions

array([2534.55, 2534.55, 2542.45, ..., 2230.07, 1665.2 , 1124.02])

In [48]:
df_sub = pd.DataFrame(data = predictions, columns=['Value'])

In [49]:
df_sub.head(50)

,Value
0,2534.550000
1,2534.550000
2,2542.450000
3,2542.450000
4,1877.560000
5,1877.560000
6,1095.500000
7,602.910000
8,386.140000
9,283.200000


In [50]:
df_sub.to_csv('output.csv')

In [52]:
t = pd.read_csv("output.csv")
t.head(5)

,Unnamed: 0,Value
0,0,2534.55
1,1,2534.55
2,2,2542.45
3,3,2542.45
4,4,1877.56


In [54]:
from sklearn.metrics import mean_squared_error 
realVals = t
predictedVals = t.Value
mse = mean_squared_error(realVals, predictedVals)
mse

ValueError: y_true and y_pred have different number of output (2!=1)

In [225]:
rmse = mean_squared_error(realVals, predictedVals, squared = False)
rmse

540.6283905122896